In [84]:
import numpy as np
import pandas as pd
import regex as re

In [85]:
shopee_data = pd.read_csv('shopee_final_data.csv')

In [86]:
shopee_data

,Product Name,Product Rating,Product Num Review,Product Num Sold,Price,Product Num Stock,Product Category,Product Specs,Store Name,Store Total Num Review,Store Total Num Product,Store Response Rate,Store Response Duration,Store Location,Produk Link
0,Xiaomi Redmi Note 9 (6GB+128GB) Onyx Black 48M...,4.9,"3,6RB","5,9RB",2399000.0,289.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Redmi Note 9'...",xiaomi.official.id,"549,3RB",190,100%,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Xiaomi-Redmi-Note-9-(6GB-...
1,"Poco X3 NFC (8GB+128GB) Snapdragon™ 732G, Laya...",4.9,"22,7RB",10RB+,3299000.0,613.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Kamera Depan': '20 MP', '...",xiaomi.official.id,"549,3RB",190,100%,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Poco-X3-NFC-(8GB-128GB)-S...
2,TERLARIS!! POCO M3 RAM 4/64 & 6/128 GARANSI RE...,5.0,485,649,2198000.0,151.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Poco M3', 'Ka...",borneo_cell88,"1,4RB",23,100%,hitungan menit,"KOTA BOGOR - TANAH SEREAL, JAWA BARAT, ID",https://shopee.co.id/TERLARIS!!-POCO-M3-RAM-4-...
3,"OPPO Reno5 8GB/128GB [50W SuperVOOC, Quad Came...",4.9,1RB,"1,5RB",4999000.0,42.0,Handphone & Tablet,"{'Merek': 'Oppo', 'Model HP': 'OPPO Reno5', 'K...",oppo.id,"125,2RB",43,100%,hitungan jam,"KOTA TANGERANG - CIPONDOH, BANTEN, ID",https://shopee.co.id/OPPO-Reno5-8GB-128GB-50W-...
4,Vivo Y20 4/64 RAM 4GB ROM 64GB Garansi Resmi O...,5.0,388,576,2059000.0,967.0,Handphone & Tablet,"{'Merek': 'Vivo', 'Model HP': 'vivo y20', 'Kap...",4aorrp07xj,"6,6RB",39,100%,hitungan menit,"KOTA JAKARTA PUSAT - SAWAH BESAR, DKI JAKARTA, ID",https://shopee.co.id/Vivo-Y20-4-64-RAM-4GB-ROM...


In [87]:
shopee_final_data_copy = shopee_data.copy()
shopee_data_copy = shopee_final_data_copy.replace({np.nan: None})

In [88]:
def search_unit(str_data):
  # only accept unit at least 2 char and max 3 char
  pattern = '(\D{2,3})'
  unit_found = re.search(pattern, str_data) 
  unit_found = list(unit_found.groups())[0] if unit_found != None else None
  return unit_found

def convert_to_numeric(string_data):
  if string_data != None:
    string_list = []
    string_list[:0] = string_data
    for idx, x in enumerate(string_list):
      if x == ',':
        string_list[idx] = '.'
    string_data = ''.join(string_list)
    string_unit = search_unit(string_data)
    if string_unit != None:
      string_unit = string_unit.lower() 
      len_unit = len(string_unit) 
      string_value = float(string_data[:-len_unit]) 
      if string_unit[:2] == 'rb':
        string_value = string_value*1000
      if string_unit[:2] == 'jt':
        string_value = string_value*1000000
    else:
      string_value = float(string_data)
  else:
    string_value = string_data
  return string_value

def search_spec_data(data_dict, key_searched):
  if data_dict != None:
    data_dict = eval(re.search(r"\{(.*?)\}", data_dict).group(0))
    if key_searched in data_dict.keys():
      return data_dict[key_searched]
    else:
      return None 
  else:
    return None

In [89]:
shopee_final_data_copy['Product Specs'][0]

"{'Merek': 'Xiaomi', 'Model HP': 'Redmi Note 9', 'Kapasitas': '128GB', 'Tipe Kartu SIM': 'Nano', 'Kamera Depan': '48MP', 'Kamera Belakang': '-', 'Tipe Garansi': 'Garansi Resmi', 'Periode Garansi': '15 Bulan', 'Ukuran Layar': '6.53', 'RAM': '6GB', 'Warna': 'Onyx Black', 'Stok': '289', 'Dikirim Dari': 'KOTA SURABAYA'}"

In [90]:
# CUSTOM FEATURES (from 'Product Specs' column)
shopee_final_data_copy['Product Merk'] = shopee_final_data_copy['Product Specs'].apply(search_spec_data, key_searched='Merek')
shopee_final_data_copy['Main Camera'] = shopee_final_data_copy['Product Specs'].apply(search_spec_data, key_searched='Kamera Belakang')
shopee_final_data_copy['Front Camera'] = shopee_final_data_copy['Product Specs'].apply(search_spec_data, key_searched='Kamera Depan')
shopee_final_data_copy['RAM'] = shopee_final_data_copy['Product Specs'].apply(search_spec_data, key_searched='RAM')
shopee_final_data_copy['Storage'] = shopee_final_data_copy['Product Specs'].apply(search_spec_data, key_searched='Kapasitas')
shopee_final_data_copy['Warranty Type'] = shopee_final_data_copy['Product Specs'].apply(search_spec_data, key_searched='Tipe Garansi')

In [91]:
shopee_final_data_copy

,Product Name,Product Rating,Product Num Review,Product Num Sold,Price,Product Num Stock,Product Category,Product Specs,Store Name,Store Total Num Review,Store Total Num Product,Store Response Rate,Store Response Duration,Store Location,Produk Link,Product Merk,Main Camera,Front Camera,RAM,Storage,Warranty Type
0,Xiaomi Redmi Note 9 (6GB+128GB) Onyx Black 48M...,4.9,"3,6RB","5,9RB",2399000.0,289.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Redmi Note 9'...",xiaomi.official.id,"549,3RB",190,100%,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Xiaomi-Redmi-Note-9-(6GB-...,Xiaomi,-,48MP,6GB,128GB,Garansi Resmi
1,"Poco X3 NFC (8GB+128GB) Snapdragon™ 732G, Laya...",4.9,"22,7RB",10RB+,3299000.0,613.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Kamera Depan': '20 MP', '...",xiaomi.official.id,"549,3RB",190,100%,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Poco-X3-NFC-(8GB-128GB)-S...,Xiaomi,64 MP,20 MP,8GB,None,Garansi Resmi
2,TERLARIS!! POCO M3 RAM 4/64 & 6/128 GARANSI RE...,5.0,485,649,2198000.0,151.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Poco M3', 'Ka...",borneo_cell88,"1,4RB",23,100%,hitungan menit,"KOTA BOGOR - TANAH SEREAL, JAWA BARAT, ID",https://shopee.co.id/TERLARIS!!-POCO-M3-RAM-4-...,Xiaomi,48,8,6GB,128GB,Garansi Resmi
3,"OPPO Reno5 8GB/128GB [50W SuperVOOC, Quad Came...",4.9,1RB,"1,5RB",4999000.0,42.0,Handphone & Tablet,"{'Merek': 'Oppo', 'Model HP': 'OPPO Reno5', 'K...",oppo.id,"125,2RB",43,100%,hitungan jam,"KOTA TANGERANG - CIPONDOH, BANTEN, ID",https://shopee.co.id/OPPO-Reno5-8GB-128GB-50W-...,Oppo,64MP + 8MP + 2MP + 2MP,44MP,8GB,128GB,Garansi Resmi
4,Vivo Y20 4/64 RAM 4GB ROM 64GB Garansi Resmi O...,5.0,388,576,2059000.0,967.0,Handphone & Tablet,"{'Merek': 'Vivo', 'Model HP': 'vivo y20', 'Kap...",4aorrp07xj,"6,6RB",39,100%,hitungan menit,"KOTA JAKARTA PUSAT - SAWAH BESAR, DKI JAKARTA, ID",https://shopee.co.id/Vivo-Y20-4-64-RAM-4GB-ROM...,Vivo,13MP+2MP+2MP,8MP,4GB,64GB,Garansi Resmi


In [92]:
# convert string to numerical value
shopee_final_data_copy['Product Num Review'] = shopee_final_data_copy['Product Num Review'].apply(convert_to_numeric)
shopee_final_data_copy['Product Num Sold'] = shopee_final_data_copy['Product Num Sold'].apply(convert_to_numeric)
shopee_final_data_copy['Store Total Num Review'] = shopee_final_data_copy['Store Total Num Review'].apply(convert_to_numeric)

In [93]:
shopee_final_data_copy

,Product Name,Product Rating,Product Num Review,Product Num Sold,Price,Product Num Stock,Product Category,Product Specs,Store Name,Store Total Num Review,Store Total Num Product,Store Response Rate,Store Response Duration,Store Location,Produk Link,Product Merk,Main Camera,Front Camera,RAM,Storage,Warranty Type
0,Xiaomi Redmi Note 9 (6GB+128GB) Onyx Black 48M...,4.9,3600.0,5900.0,2399000.0,289.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Redmi Note 9'...",xiaomi.official.id,549300.0,190,100%,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Xiaomi-Redmi-Note-9-(6GB-...,Xiaomi,-,48MP,6GB,128GB,Garansi Resmi
1,"Poco X3 NFC (8GB+128GB) Snapdragon™ 732G, Laya...",4.9,22700.0,10000.0,3299000.0,613.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Kamera Depan': '20 MP', '...",xiaomi.official.id,549300.0,190,100%,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Poco-X3-NFC-(8GB-128GB)-S...,Xiaomi,64 MP,20 MP,8GB,None,Garansi Resmi
2,TERLARIS!! POCO M3 RAM 4/64 & 6/128 GARANSI RE...,5.0,485.0,649.0,2198000.0,151.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Poco M3', 'Ka...",borneo_cell88,1400.0,23,100%,hitungan menit,"KOTA BOGOR - TANAH SEREAL, JAWA BARAT, ID",https://shopee.co.id/TERLARIS!!-POCO-M3-RAM-4-...,Xiaomi,48,8,6GB,128GB,Garansi Resmi
3,"OPPO Reno5 8GB/128GB [50W SuperVOOC, Quad Came...",4.9,1000.0,1500.0,4999000.0,42.0,Handphone & Tablet,"{'Merek': 'Oppo', 'Model HP': 'OPPO Reno5', 'K...",oppo.id,125200.0,43,100%,hitungan jam,"KOTA TANGERANG - CIPONDOH, BANTEN, ID",https://shopee.co.id/OPPO-Reno5-8GB-128GB-50W-...,Oppo,64MP + 8MP + 2MP + 2MP,44MP,8GB,128GB,Garansi Resmi
4,Vivo Y20 4/64 RAM 4GB ROM 64GB Garansi Resmi O...,5.0,388.0,576.0,2059000.0,967.0,Handphone & Tablet,"{'Merek': 'Vivo', 'Model HP': 'vivo y20', 'Kap...",4aorrp07xj,6600.0,39,100%,hitungan menit,"KOTA JAKARTA PUSAT - SAWAH BESAR, DKI JAKARTA, ID",https://shopee.co.id/Vivo-Y20-4-64-RAM-4GB-ROM...,Vivo,13MP+2MP+2MP,8MP,4GB,64GB,Garansi Resmi


In [94]:
def clean_response_rate(response_rate_data):
  resp_rate = float(response_rate_data.split('%')[0]) if response_rate_data != None else None
  return resp_rate

def clean_mpgb(mpgb_data):
  mpgb_data = float(mpgb_data[:-2]) if mpgb_data != None else None
  return mpgb_data

def clean_multi_mpgb(multi_mpgb_data):
  if multi_mpgb_data != None:
    multi_mpgb_data = re.findall('[0-9]{1,3}', multi_mpgb_data)
    total_mpgb = 0
    if len(multi_mpgb_data) > 0:
      for x in multi_mpgb_data:
        total_mpgb += float(x)
    return total_mpgb
  else:
    return None

In [95]:
shopee_final_data_copy['Store Response Rate'] = shopee_final_data_copy['Store Response Rate'].apply(clean_response_rate)
shopee_final_data_copy['Main Camera'] = shopee_final_data_copy['Main Camera'].apply(clean_multi_mpgb)
shopee_final_data_copy['Front Camera'] = shopee_final_data_copy['Front Camera'].apply(clean_multi_mpgb)
shopee_final_data_copy['RAM'] = shopee_final_data_copy['RAM'].apply(clean_multi_mpgb)
shopee_final_data_copy['Storage'] = shopee_final_data_copy['Storage'].apply(clean_multi_mpgb)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True


In [96]:
shopee_final_data_copy

,Product Name,Product Rating,Product Num Review,Product Num Sold,Price,Product Num Stock,Product Category,Product Specs,Store Name,Store Total Num Review,Store Total Num Product,Store Response Rate,Store Response Duration,Store Location,Produk Link,Product Merk,Main Camera,Front Camera,RAM,Storage,Warranty Type
0,Xiaomi Redmi Note 9 (6GB+128GB) Onyx Black 48M...,4.9,3600.0,5900.0,2399000.0,289.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Redmi Note 9'...",xiaomi.official.id,549300.0,190,100.0,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Xiaomi-Redmi-Note-9-(6GB-...,Xiaomi,0.0,48.0,6.0,128.0,Garansi Resmi
1,"Poco X3 NFC (8GB+128GB) Snapdragon™ 732G, Laya...",4.9,22700.0,10000.0,3299000.0,613.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Kamera Depan': '20 MP', '...",xiaomi.official.id,549300.0,190,100.0,hitungan menit,KOTA SURABAYA,https://shopee.co.id/Poco-X3-NFC-(8GB-128GB)-S...,Xiaomi,64.0,20.0,8.0,NaN,Garansi Resmi
2,TERLARIS!! POCO M3 RAM 4/64 & 6/128 GARANSI RE...,5.0,485.0,649.0,2198000.0,151.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Model HP': 'Poco M3', 'Ka...",borneo_cell88,1400.0,23,100.0,hitungan menit,"KOTA BOGOR - TANAH SEREAL, JAWA BARAT, ID",https://shopee.co.id/TERLARIS!!-POCO-M3-RAM-4-...,Xiaomi,48.0,8.0,6.0,128.0,Garansi Resmi
3,"OPPO Reno5 8GB/128GB [50W SuperVOOC, Quad Came...",4.9,1000.0,1500.0,4999000.0,42.0,Handphone & Tablet,"{'Merek': 'Oppo', 'Model HP': 'OPPO Reno5', 'K...",oppo.id,125200.0,43,100.0,hitungan jam,"KOTA TANGERANG - CIPONDOH, BANTEN, ID",https://shopee.co.id/OPPO-Reno5-8GB-128GB-50W-...,Oppo,76.0,44.0,8.0,128.0,Garansi Resmi
4,Vivo Y20 4/64 RAM 4GB ROM 64GB Garansi Resmi O...,5.0,388.0,576.0,2059000.0,967.0,Handphone & Tablet,"{'Merek': 'Vivo', 'Model HP': 'vivo y20', 'Kap...",4aorrp07xj,6600.0,39,100.0,hitungan menit,"KOTA JAKARTA PUSAT - SAWAH BESAR, DKI JAKARTA, ID",https://shopee.co.id/Vivo-Y20-4-64-RAM-4GB-ROM...,Vivo,17.0,8.0,4.0,64.0,Garansi Resmi


In [97]:
shopee_final_data_cleaned = pd.DataFrame.from_dict(shopee_final_data_copy)
shopee_final_data_cleaned.to_csv('shopee_final_data_cleaned.csv', index=False)